In [2]:
!pip install yfinance
!pip install yahooquery
from yahooquery import Screener
import yfinance as yf   
import pandas as pd
import numpy as np
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense,Dropout
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
import tensorflow as tf
import random 
from tensorflow.keras.models import load_model


In [3]:
clmns = [
    "Open 12-11","Open 11-10","Open 10-9","Open 9-8","Open 8-7","Open 7-6","Open 6-5","Open 5-4","Open 4-3","Open 3-2","Open 2-1",
    "Close 12-11","Close 11-10","Close 10-9","Close 9-8","Close 8-7","Close 7-6","Close 6-5","Close 5-4","Close 4-3","Close 3-2","Close 2-1",
    "High 12-11","High 11-10","High 10-9","High 9-8","High 8-7","High 7-6","High 6-5","High 5-4","High 4-3","High 3-2","High 2-1",
    "Low 12-11","Low 11-10","Low 10-9","Low 9-8","Low 8-7","Low 7-6","Low 6-5","Low 5-4","Low 4-3","Low 3-2","Low 2-1",
    "AdjV 12-11","AdjV 11-10","AdjV 10-9","AdjV 9-8","AdjV 8-7","AdjV 7-6","AdjV 6-5","AdjV 5-4","AdjV 4-3","AdjV 3-2","AdjV 2-1",
    "Volume 12-11","Volume 11-10","Volume 10-9","Volume 9-8","Volume 8-7","Volume 7-6","Volume 6-5","Volume 5-4","Volume 4-3","Volume 3-2","Volume 2-1",
    "suggestion"
]

def read_syms_from_txt():
  with open("syms.txt", "r") as f:
    lines = f.readlines()
    f.close()
  lst = []
  for line in lines:
    line = line.strip()
    lst.append(line)
  symbols = lst
  return symbols
def get_crypto_syms():
   # 'all_cryptocurrencies_au','all_cryptocurrencies_ca','all_cryptocurrencies_eu','all_cryptocurrencies_gb','all_cryptocurrencies_in',
   screens = [
       'all_cryptocurrencies_us', 'all_cryptocurrencies_au', 'all_cryptocurrencies_ca', 'all_cryptocurrencies_eu', 'all_cryptocurrencies_gb', 'all_cryptocurrencies_in', ]
   s = Screener()
   symbols = []
   for i in screens:
      data = s.get_screeners(i, count=250)
      dicts = data[i]['quotes']
      syms = [d['symbol'] for d in dicts]
      for sym in syms:
        symbols.append(sym)
   # print(len(symbols))
   # pieces = 15
   # new_arrays = np.array_split(symbols, pieces)
   return symbols
def spliting(data):
  X = data.drop(["yes","no"], axis=1)
  y = data[["yes","no"]]
  xTrain, xTest, yTrain, yTest = train_test_split(X, y, test_size=0.1)
  print(xTrain.shape, end=" ")
  print(yTrain.shape)
  print(xTest.shape, end=" ")
  print(yTest.shape)
  return xTrain, xTest, yTrain, yTest
def scaler(row):
    scaler = MinMaxScaler(feature_range=(-3, 3))
    row = scaler.fit_transform(row)
    return row
def process(data):
    data = data.dropna()
    row = []
    data = np.array(data)
    llst = [0, 1, 2, 3, 4, 5]

    for i in range(12, data.shape[0]):
        grow = []
        srow = []

        for j in llst:

           srow.append([
               data[i-1][j] - data[i-2][j],
               data[i-2][j] - data[i-3][j],
               data[i-3][j] - data[i-4][j],
               data[i-4][j] - data[i-5][j],
               data[i-5][j] - data[i-6][j],
               data[i-6][j] - data[i-7][j],
               data[i-7][j] - data[i-8][j],
               data[i-8][j] - data[i-9][j],
               data[i-9][j] - data[i-10][j],
               data[i-10][j] - data[i-11][j],
               data[i-11][j] - data[i-12][j]
           ])

        for lst in srow:
            mm = np.array(lst)
            mm = np.reshape(mm, (-1, 1))
            grow.append(scaler(mm))

        sugg = "no"
        if data[i][3] > data[i][0]:
            sugg = "yes"

        arr = np.array(grow).flatten()
        arr = np.append(arr, sugg)
        row.append(arr)


    grow = []
    srow = []
    llst = []
    data = []
    arr = []
    mm = []

    return np.array(row)

In [ ]:
def get_data(symbols):
    unattached_dfs = []
    for symbol in symbols:
         data = yf.download(symbol, period="730d",
                            interval="1h", progress=False)
         if data.empty:
              pass
         else:
               if data.shape[0] > 12:
                   unattached_dfs.append(
                       process(data))

    data = np.array(unattached_dfs[0])
    for i in unattached_dfs[1:]:
        data = np.append(data, np.array(i), axis=0)
        
    symbol = []

    data = pd.DataFrame(data, columns=clmns)
    sugg = data["suggestion"]
    data.drop("suggestion",axis=1,inplace=True)
    sugg = pd.get_dummies(sugg)
    data = pd.concat([data,sugg],axis=1)
    data = data.astype(float)

    xTrain, xTest, yTrain, yTest = spliting(data)
    print(data.shape)
    data.to_csv(f"{random.randint(1,10000)}.csv")
    #data = []
    return xTrain, xTest, yTrain, yTest,data



#symbols = read_syms_from_txt()
symbols = get_crypto_syms()

pieces = 50
new_arrays = np.array_split(symbols, pieces)
for symbols in new_arrays[25:]:
    get_data(symbols)
#symbols = ["btc-usd","eth-usd","trx-usd","bnb-usd","xrp-usd"]
#xTrain, xTest, yTrain, yTest,data= get_data(symbols)
#data 

In [4]:
data = pd.concat([pd.read_parquet("/content/drive/MyDrive/ch1.parquet"),pd.read_parquet("/content/drive/MyDrive/ch2.parquet")])
data

,Unnamed: 0,Open 12-11,Open 11-10,Open 10-9,Open 9-8,Open 8-7,Open 7-6,Open 6-5,Open 5-4,Open 4-3,...,Volume 9-8,Volume 8-7,Volume 7-6,Volume 6-5,Volume 5-4,Volume 4-3,Volume 3-2,Volume 2-1,no,yes
0,0,0.403234,1.292086,0.162204,-0.075960,3.000000,-1.954447,-0.679597,-0.324772,-3.000000,...,0.399351,-0.367985,1.249609,0.673462,1.107995,2.759349,1.446223,0.807556,0.0,1.0
1,1,0.986168,0.403234,1.292086,0.162204,-0.075960,3.000000,-1.954447,-0.679597,-0.324772,...,3.000000,0.399351,-0.367985,1.249609,0.673462,1.107995,2.759349,1.446223,1.0,0.0
2,2,2.373060,0.986168,0.403234,1.292086,0.162204,-0.075960,3.000000,-1.954447,-0.679597,...,-3.000000,3.000000,0.399351,-0.367985,1.249609,0.673462,1.107995,2.759349,0.0,1.0
3,3,-1.501655,2.240755,0.561183,-0.144770,0.931659,-0.436665,-0.725089,3.000000,-3.000000,...,1.235703,-3.000000,3.000000,0.399351,-0.367985,1.249609,0.673462,1.107995,0.0,1.0
4,4,-0.233639,-1.501655,2.240755,0.561183,-0.144770,0.931659,-0.436665,-0.725089,3.000000,...,0.005696,0.767297,-3.000000,2.336488,0.023433,-0.659047,0.779665,0.267232,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
314612,314612,3.000000,0.066568,2.204005,0.872484,0.589718,0.974172,-0.977439,0.045995,-3.000000,...,0.000000,0.000000,0.000000,-3.000000,3.000000,0.000000,0.000000,0.000000,0.0,1.0
314613,314613,-0.396222,3.000000,0.066568,2.204005,0.872484,0.589718,0.974172,-0.977439,0.045995,...,0.000000,0.000000,0.000000,0.000000,-3.000000,3.000000,0.000000,0.000000,0.0,1.0
314614,314614,1.961979,-0.396222,3.000000,0.066568,2.204005,0.872484,0.589718,0.974172,-0.977439,...,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,3.000000,0.000000,1.0,0.0
314615,314615,1.530667,1.434136,-2.123230,3.000000,-1.425107,1.799235,-0.209376,-0.635931,-0.055978,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,-3.000000,3.000000,1.0,0.0


In [7]:
data.drop("Unnamed: 0",axis=1,inplace=True)

In [9]:
data.to_parquet("ch_1.parquet")

In [8]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 7571614 entries, 0 to 314616
Data columns (total 68 columns):
 #   Column        Dtype  
---  ------        -----  
 0   Open 12-11    float64
 1   Open 11-10    float64
 2   Open 10-9     float64
 3   Open 9-8      float64
 4   Open 8-7      float64
 5   Open 7-6      float64
 6   Open 6-5      float64
 7   Open 5-4      float64
 8   Open 4-3      float64
 9   Open 3-2      float64
 10  Open 2-1      float64
 11  Close 12-11   float64
 12  Close 11-10   float64
 13  Close 10-9    float64
 14  Close 9-8     float64
 15  Close 8-7     float64
 16  Close 7-6     float64
 17  Close 6-5     float64
 18  Close 5-4     float64
 19  Close 4-3     float64
 20  Close 3-2     float64
 21  Close 2-1     float64
 22  High 12-11    float64
 23  High 11-10    float64
 24  High 10-9     float64
 25  High 9-8      float64
 26  High 8-7      float64
 27  High 7-6      float64
 28  High 6-5      float64
 29  High 5-4      float64
 30  High 4-3      float

In [ ]:
model = Sequential()



model.add(Dense(2000, activation='relu', input_shape=(xTrain.shape[1],)))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))
model.add(Dense(1000, activation='relu'))



model.add(Dense(2, activation='sigmoid'))
model.compile(optimizer="adam", loss='binary_crossentropy',metrics=['accuracy'])
model.summary()

In [ ]:
model.fit(xTrain,yTrain,epochs=35,batch_size=4096,validation_data=(xTest,yTest))

In [12]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive
